# Demonstration of our algorithm running over a sampled instance from Gaussian process with one dimensional input.

## import relevant packages

In [1]:
import sys
sys.path.append('..')
import POP_BO.popbo.util as util
from POP_BO.popbo import POPBO
import POP_BO.popbo.Oracles as sigma
import POP_BO.popbo.NewOracles as oracles
import numpy as np
import matplotlib
matplotlib.use('qt5agg')
import matplotlib.pyplot as plt
# %load_ext autoreload
# %autoreload 1

## Construct the problem configurations

In [21]:
def matyas(x):
    x = np.atleast_2d(x)
    return -(0.26*(x[:,0]**2 + x[:,1]**2) - 0.48*x[:,0]*x[:,1])
np.random.seed(3)
config = util.get_config('test_func_Shubert', problem_dim=2)
f = config['obj']
#config["opt_val"] = 0.
#f = matyas
print(config)

{'problem_name': 'test_func_Shubert', 'numerical_epsilon': 1e-06, 'beta_func': <function get_config.<locals>.<lambda> at 0x33d92eca0>, 'norm_bound': 6, 'var_dim': 2, 'discretize_num_list': [20, 20], 'bounds': [[-5.119999, 5.119999], [-5.119999, 5.119999]], 'noise_var': 0.0, 'x_grid': array([[-5.119999  , -5.119999  ],
       [-4.58105174, -5.119999  ],
       [-4.04210447, -5.119999  ],
       [-3.50315721, -5.119999  ],
       [-2.96420995, -5.119999  ],
       [-2.42526268, -5.119999  ],
       [-1.88631542, -5.119999  ],
       [-1.34736816, -5.119999  ],
       [-0.80842089, -5.119999  ],
       [-0.26947363, -5.119999  ],
       [ 0.26947363, -5.119999  ],
       [ 0.80842089, -5.119999  ],
       [ 1.34736816, -5.119999  ],
       [ 1.88631542, -5.119999  ],
       [ 2.42526268, -5.119999  ],
       [ 2.96420995, -5.119999  ],
       [ 3.50315721, -5.119999  ],
       [ 4.04210447, -5.119999  ],
       [ 4.58105174, -5.119999  ],
       [ 5.119999  , -5.119999  ],
       [-5.1199

## Initialize the algorithm

In [22]:
popbo = POPBO(config, y_lb=-5, y_ub=5)
x_prime = [np.random.rand(), np.random.rand()]
x = popbo.query_new_point(x_prime=x_prime)

  index  |  Mat52.lengthscale  |  constraints  |  priors
  [0]    |         0.80000000  |      +ve      |        
  [1]    |         0.80000000  |      +ve      |        
[-5.1193387  -5.10791188] with opt improvement 8.48528342097763, real 8.485285542289919


## repeatedly and interatively run the following block to see how our algorithm works

In [28]:
plot_uncer = False  # set to false if you do not want to plot to save time
sample_per_run = 50
#oracle = oracles.get_named_oracle("matyas", deterministic=False)
oracle = lambda x, x_prime: oracles.det_oracle(f(x), f(x_prime))
#oracle = lambda x, x_prime: sigma.pref_oracle(f(x), f(x_prime))
for k in range(sample_per_run):
    x_prime = x
    x = popbo.query_new_point(x_prime=x_prime)
    one = oracle(x, x_prime)
    popbo.add_new_data(x_prime=x_prime, x=x, one=one)
    print(f'New sampled point: {x}, with objective value {f(x)}.')

[-3.4788202   4.03395568] with opt improvement 6.217081213150088, real 6.217083264658573
Estimated best x: [-3.6374204518945605, 4.1331154236958145],  with regret 6.11898207308409.
Step 51: norm 6.000000028389237, LL value -17.450471595466382,avg -0.3421661097150271.
New sampled point: [-3.478820198026647, 4.033955675325178], with objective value 0.12707424419752356.
[-4.10531687  4.87483738] with opt improvement 3.310352241065565, real 3.31035461831144
Estimated best x: [-3.6374204518945605, 4.1331154236958145],  with regret 6.11898207308409.
Step 52: norm 6.0000000283405575, LL value -18.455886721862555,avg -0.3549208984973568.
New sampled point: [-4.105316865551445, 4.87483738054712], with objective value 1.0063816822176352.
[-3.86357788  3.55081569] with opt improvement 4.125960509250241, real 4.125962707831778
Estimated best x: [-3.6374204518945605, 4.1331154236958145],  with regret 6.11898207308409.
Step 53: norm 6.000000028400421, LL value -18.68421079572269,avg -0.3525322791645

In [28]:
if plot_uncer:
    num_data = popbo.num_data
    x_grid = popbo.config['x_grid']
    max_ll_list = []
    upb_list = []
    lpb_list = []
    f_list = []
    for x in x_grid:
        max_ll_list.append(popbo.get_maxLL_min_norm_inter(x))
        upb_list.append(popbo.get_upper_bound(x, beta=0.2*np.sqrt(num_data)))
        lpb_list.append(popbo.get_lower_bound(x, beta=0.2*np.sqrt(num_data)))
        f_list.append(f(x))
    sampled_f_list = []
    for x in popbo.x_list:
        sampled_f_list.append(f(x))

    fig= plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.plot(x_grid[:,0], x_grid[:,1], f_list.reshape(x_grid.shape))
    ax.plot(x_grid[:,0], x_grid[:,1], max_ll_list.reshape(x_grid.shape))
    ax.plot(x_grid[:,0], x_grid[:,1], upb_list.reshape(x_grid.shape))
    ax.plot(x_grid[:,0], x_grid[:,1], lpb_list.reshape(x_grid.shape))
    ax.scatter(popbo.x_list, sampled_f_list, marker='x')
    ax.legend(['Ground truth', 'Maximum Likelihood Estimator', 'Confidence Upper Bound', 'Confidence Lower Bound', 'Sampled Points'])
    plt.show()



## plot the regret evolution

In [29]:
x_list = [popbo.x_prime_list[0]] + popbo.x_list
regret = [abs(popbo.config['opt_val'] - f(x)) for x in x_list]
fig, ax  = plt.subplots()
ax.plot(np.cumsum(regret))
ax.plot([0, popbo.num_data], [regret[0], sum(regret)])
ax.legend(['cumulative regret', 'linear reference'])
plt.show()
#plt.plot(opbo.estimated_best_x_list)

In [30]:
best_x = popbo.estimated_best_x_list
best_regret = [abs(popbo.config['opt_val'] - f(x)) for x in best_x]
fig, ax  = plt.subplots()
ax.plot(np.cumsum(regret))
ax.plot([0, popbo.num_data], [regret[0], sum(regret)])
ax.legend(['cumulative regret', 'linear reference'])
plt.show()

In [31]:
popbo.config["opt_val"] - f(best_x[-1])

5.404252167860172

In [32]:
import os.path as path
